In [ ]:
import fastkaggle
import polars as pl

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import cross_validate, KFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

In [ ]:
comp = "playground-series-s4e2"
path = fastkaggle.setup_comp(comp)

train = pl.read_csv(path/"train.csv").drop("id").with_columns(pl.col(pl.Float64).cast(pl.Float32))
test = pl.read_csv(path/"test.csv").drop("id").with_columns(pl.col(pl.Float64).cast(pl.Float32))
subm = pl.read_csv(path/"sample_submission.csv") 

In [ ]:
cat_cols = train.drop("NObeyesdad").select(pl.col(pl.String)).columns
num_cols = train.select(pl.col(pl.Float32)).columns

In [ ]:
X = train.drop("NObeyesdad")
ord_enc = OrdinalEncoder()
y = ord_enc.fit_transform(train.select(pl.last())).flatten()

pipe = make_column_transformer(
    (OneHotEncoder(sparse_output=False, handle_unknown="ignore"), cat_cols),
    (StandardScaler(), num_cols),
    remainder="passthrough", verbose_feature_names_out=False
).set_output(transform="polars")

cv = KFold()
model = HistGradientBoostingClassifier()
mod_pipe = make_pipeline(pipe, model)
cross_validate(mod_pipe,X,y,cv=cv,scoring="accuracy")

In [ ]:
ord_enc.categories_

In [ ]:
ord_enc.inverse_transform(np.array([1,3]).reshape(-1,1))

In [ ]:
mod_pipe.fit(X,y)
preds = mod_pipe.predict(test)

In [ ]:
preds_str = ord_enc.inverse_transform(preds.reshape(-1,1)).squeeze()

In [ ]:

ss = subm.with_columns("id", NObeyesdad=preds_str)
ss.write_csv(path/"subm.csv")

In [ ]:
from kaggle import api
api.competition_submit(path/"subm.csv", "routince boost tree", comp)